In [1]:
import pandas as pd

<h1>UsersWorstDeveloper</h1>

def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [2]:
# Cargamos los archivos CSV a utilizar:
games_df = pd.read_csv(r"..\Archivos Producidos\games.csv")
reviews_df = pd.read_csv(r"..\Archivos Producidos\reviews.csv")

# Convertimos algunos tipos de columna que no se cargaron correctamente:
reviews_df["item_id"] = reviews_df["item_id"].astype("Int64")
reviews_df["year"] = reviews_df["year"].astype("Int64")

In [3]:
games_df.dtypes

app_name                     object
release_year                  int64
item_id                       int64
developer                    object
Action                        int64
Adventure                     int64
Animation &amp; Modeling      int64
Audio Production              int64
Casual                        int64
Design &amp; Illustration     int64
Early Access                  int64
Education                     int64
Free to Play                  int64
Indie                         int64
Massively Multiplayer         int64
Photo Editing                 int64
RPG                           int64
Racing                        int64
Simulation                    int64
Software Training             int64
Sports                        int64
Strategy                      int64
Utilities                     int64
Video Production              int64
Web Publishing                int64
dtype: object

In [4]:
reviews_df.dtypes

user_id               object
item_id                Int64
recommend               bool
review                object
year                   Int64
sentiment_analysis     int64
dtype: object

Ya cargados los archivos, comenzamos con las transformaciones:

In [5]:
# Unimos las columnas que nos serán de utilidad:
df_r_g = pd.merge(games_df[["developer", "item_id"]], reviews_df[["item_id", "recommend", "year", "sentiment_analysis"]], on="item_id")
df_r_g.head()

,developer,item_id,recommend,year,sentiment_analysis
0,Stainless games ltd,282010,True,<NA>,1
1,Valve,70,True,2015,1
2,Valve,70,True,2011,1
3,Valve,70,True,2014,1
4,Valve,70,True,2013,2


In [6]:
print(len(df_r_g)) # Chequeamos la cantidad de registros

48832


In [7]:
# Eliminamos registros donde "recommend" no sea False:
df_r_g = df_r_g[df_r_g["recommend"] == False]

# Eliminamos registros donde "sentiment_analysis" no sea 0:
df_r_g = df_r_g[df_r_g["sentiment_analysis"] == 0]

# Eliminamos registros donde "year" sea nulo:
df_r_g = df_r_g[df_r_g["year"] != None]

In [8]:
print(len(df_r_g)) # Chequeamos la cantidad de registros

1325


In [9]:
# Eliminamos la columnas que ya no utilizaremos:
df_r_g.drop(columns=["item_id", "recommend", "sentiment_analysis"], inplace=True)
df_r_g.head()

,developer,year
73,Outerlight ltd.,2014
91,Introversion software,2014
141,Facepunch studios,2015
246,Facepunch studios,2015
371,Facepunch studios,2015


In [10]:
# Agregamos una columna de no-recomendaciones:
df_r_g["no_recommendation"] = 1

# Agrupamos por "developer" y "year" y contamos las no-recomendaciones:
df_usersworstdeveloper = df_r_g.groupby(["developer", "year"]).agg({"no_recommendation": "count"}).reset_index()

# Lo ordenamos en cantidad de no-recomendaciones descendiente:
df_usersworstdeveloper = df_usersworstdeveloper.sort_values(by="no_recommendation", ascending=False)

df_usersworstdeveloper.head()

,developer,year,no_recommendation
548,Valve,2015,58
62,Bohemia interactive,2015,34
203,Facepunch studios,2015,29
547,Valve,2014,28
61,Bohemia interactive,2014,27


Desarrollamos la función:

In [11]:
# Creamos la función UsersWorstDeveloper:
def UsersWorstDeveloper(año):

    # Corroboramos que el año introducido sea un número:
    if isinstance(año, int):
        
        # En caso de que el año no se encuentre en el DataFrame:
        if año not in df_usersworstdeveloper["year"].tolist(): return f"{año} no se encuentra entre los años disponibles."

        # Caso contrario (ideal):
        else:

            # Filtramos por año:
            df_año = df_usersworstdeveloper[df_usersworstdeveloper["year"] == año].head(3)

            # Creamos la variable que utilizaremos como respuesta final:
            respuesta = [{"Puesto 1": None}, {"Puesto 2": None}, {"Puesto 3": None}]

            # Rellenamos esa variable:
            respuesta[0]["Puesto 1"] = df_año.iloc[0]["developer"]
            respuesta[1]["Puesto 2"] = df_año.iloc[1]["developer"]
            respuesta[2]["Puesto 3"] = df_año.iloc[2]["developer"]

            # Devolvemos la respuesta final:
            return respuesta


    # En caso de que el año introducido no sea un número:
    else: return f"'{año}' no es un año válido."

In [12]:
# Probamos la función
print(UsersWorstDeveloper("Prueba"))
print(UsersWorstDeveloper( 2015))
print(UsersWorstDeveloper(  2018 ))
print(UsersWorstDeveloper(2018))
print(UsersWorstDeveloper(2010))
print(UsersWorstDeveloper(2013 ))

'Prueba' no es un año válido.
[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Bohemia interactive'}, {'Puesto 3': 'Facepunch studios'}]
2018 no se encuentra entre los años disponibles.
2018 no se encuentra entre los años disponibles.
2010 no se encuentra entre los años disponibles.
[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Infinity ward'}, {'Puesto 3': 'Jagex limited'}]


Guardamos el DataFrame utilizado:

In [13]:
# Guardamos el dataframe "df_usersworstdeveloper":
df_usersworstdeveloper.to_csv(r"..\Archivos Producidos\Data para Endpoints\df_usersworstdeveloper.csv", index=False)